In [1]:
import cv2
import numpy as np

# Specify the full paths to the weight and config files
weights_path = r"D:\SIH\yolov3.weights"
config_path = r"D:\SIH\yolov3.cfg"

# Load YOLO
net = cv2.dnn.readNet(weights_path, config_path)
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Load the image
image_path = r"D:\SIH\traffic.jpg"  # Updated path to your image
img = cv2.imread(image_path)

# Check if image is loaded properly
if img is None:
    raise ValueError(f"Image at path {image_path} could not be loaded.")

height, width, channels = img.shape

# Prepare the image for the neural network
blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
net.setInput(blob)
outs = net.forward(output_layers)

# Initialize lists to hold the detection data
class_ids = []
confidences = []
boxes = []

# Parse the network's output
for out in outs:
    for detection in out:
        scores = detection[5:]  # Get the scores for the detected classes
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:  # Filter out weak detections
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)

            # Calculate the coordinates of the bounding box
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)

            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

# Apply non-maxima suppression to remove overlapping boxes
indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

# Check if indices is empty
if len(indices) == 0:
    print("No objects detected.")
else:
    # Handle different cases for `indices`
    if isinstance(indices, tuple):
        indices = indices[0]

    # Ensure indices is a numpy array
    if isinstance(indices, np.ndarray):
        indices = indices.flatten()

    # Draw bounding boxes on the image
    for i in indices:
        box = boxes[i]
        x, y, w, h = box[0], box[1], box[2], box[3]
        label = str(class_ids[i])
        confidence = confidences[i]
        color = (0, 255, 0)  # Green color for bounding boxes
        thickness = 2  # Thickness of the bounding box lines
        cv2.rectangle(img, (x, y), (x + w, y + h), color, thickness)
        cv2.putText(img, f'{label} {confidence:.2f}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, thickness)

# Save the image with detections to check results
output_path = r"D:\SIH\output_image.jpg"
cv2.imwrite(output_path, img)
print(f"Image saved to {output_path}")

# Display the image with detections
cv2.imshow("Image", img)
cv2.waitKey(20000)  # Wait for 2 seconds before closing the window
cv2.destroyAllWindows()

Image saved to D:\SIH\output_image.jpg
